In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline


## running the model

In [3]:
from Regard import init
args = init(verbose=1, epochs = 30)
print('Parameters=', args)

path = 'Regard.pt'

Parameters= {'batch_size': 8, 'test_batch_size': 400, 'epochs': 30, 'lr': 0.01, 'momentum': 0.25, 'no_cuda': True, 'num_processes': 1, 'seed': 42, 'log_interval': 10, 'dimension': 50, 'verbose': 1, 'conv1_dim': 10, 'conv1_kernel_size': 5, 'conv2_dim': 20, 'conv2_kernel_size': 5, 'crop': 181, 'size': 128, 'mean': 0.3, 'std': 0.6}


In [4]:
from Regard import ML
ml = ML(args)

import os
import torch
ml.model.load_state_dict(torch.load(path))
print('Loading file', path)



cuda? False
cuda? False
Loading file Regard.pt


## testing the model

In [5]:
import imageio
image = imageio.imread('./dataset/right/2018-03-20_Nicolas_40.png')
image = (image-image.min())/(image.max()-image.min()) * 1.
image *= args.std
image += args.mean
print('size of input image', image.shape)

size of input image (243, 243, 3)


In [6]:
import numpy as np
data = torch.from_numpy(np.transpose(image, (2, 0, 1)))
#data = torch.from_numpy(image)
print('size of input image', data.shape)
from torchvision import transforms


size of input image torch.Size([3, 243, 243])


In [7]:
#t = transforms.Normalize(mean=args.mean, std=args.std)
#data = t(data)

In [8]:
import time
t0 = time.time()
#output = ml.model(t(data).unsqueeze_(0))
output = ml.model(data.unsqueeze_(0)).double()
pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
print('Elapsed time =', (time.time()-t0)*1000, 'ms')
print('Prediction =', ml.d.classes[pred])

RuntimeError: Expected object of type torch.DoubleTensor but found type torch.FloatTensor for argument #2 'weight'

In [9]:
image = imageio.imread('./dataset/right/2018-03-20_Nicolas_40.png')
t = transforms.Normalize(mean=[args.mean]*3, std=[args.std]*3)

def classify(image):
    data = torch.from_numpy(image)
    output = ml.model(t(data).unsqueeze_(0))
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    return pred

t0 = time.time()
classify(image)
print('Elapsed time =', (time.time()-t0)*1000, 'ms')
print('Prediction =', ml.d.classes[pred])

# dynamical test

In [ ]:

from LeCheapEyeTracker.EyeTrackerServer import Server
from LeCheapEyeTracker.EyeTrackerClient import Client

N_category = len(targets.keys())
def dyn_test(N_frame=64, startup_time=1., interframe_time=1., waiting_time=.2):
    try:
        import time
        time.sleep(startup_time)
        et = Server()
        print ('Time', et.clock())
        img0 = et.cam.grab()
        img_face, res, t0 = et.process_frame(img0, et.clock())

        H, W, three = img_face.shape
        #movie = np.zeros((H, W, 3, N_frame), dtype=np.uint8)
        for i in range(N_frame):            
            # presentation of stimulus
            time.sleep(interframe_time)

            frame = et.cam.grab()
            img_face, res, t0 = et.process_frame(frame, et.clock())

            pred = classify(image)
            print('Prediction =', ml.d.classes[pred])
            time.sleep(waiting_time)

    finally:
        et.close()
        
dyn_test()        

In [ ]:
(128 - 12) **2 * 20 / 16

In [ ]:
16820 

In [ ]:
import numpy as np
np.sqrt(841)

In [ ]:
126 / 4

In [ ]:
4 * 29

In [ ]:
padding = 2* (2* (args.conv1_kernel_size -1)//2 + (args.conv2_kernel_size -1)//2)
fc1_dim = int((args.size - padding) **2 * args.conv2_dim / 16)
        

In [ ]:
fc1_dim

In [ ]:
padding